In [1]:
import pandas as pd
import transformers
import spacy
import seaborn as sns

In [2]:
fwd = pd.read_csv('trustpilot_reviews.csv', index_col=0)
fwd

,title,delay,date,rating,review,user,page,entities,sentiment
0,MAID INSURANCE,20 hours ago,2022-09-10,Rated 1 out of 5 stars,NOT USER FRIENDLY,631ac3fc499c0f001378347b,1,(),0.0
1,Car Renewal Insurance,A day ago,2022-09-09,Rated 5 out of 5 stars,Smooth transaction via online renewal,631a0595460e3e0013d01cf3,1,(),0.0
2,Efficient and easy,2 days ago,2022-09-08,Rated 5 out of 5 stars,NaN,6319d123804e6a0012378a4c,1,(),0.0
3,user friendly,2 days ago,2022-09-08,Rated 5 out of 5 stars,NaN,63194cf0a09d850012fec379,1,(),0.0
4,Simple & User Friendly Platform,2 days ago,2022-09-08,Rated 5 out of 5 stars,It was straight forward and the discount given...,6319460f9c69470012db410d,1,"(Malaysia, FWD, FWD)",0.0
...,...,...,...,...,...,...,...,...,...
19184,The renewal was a breeze,"Jul 14, 2017",2017-07-14,Rated 5 out of 5 stars,NaN,5968abc80000ff000aaf4824,976,(),0.0
19185,Hi Guys,"Jul 14, 2017",2017-07-14,Rated 4 out of 5 stars,"Hi Guys, you did well, area to fine tune on ca...",5968a4aa0000ff000aaf453d,976,(),0.0
19186,I am quite happy with the service and …,"Jul 14, 2017",2017-07-14,Rated 5 out of 5 stars,I am quite happy with the service and premium ...,596887300000ff000aaf3a17,976,"(FWD, year end)",0.0
19187,I recently purchased an insurance for my FDW.,"Jul 14, 2017",2017-07-14,Rated 5 out of 5 stars,I recently purchased an insurance policy for m...,596886320000ff000aaf39c0,976,(),0.0


In [29]:
# extract entities from reviews
nlp = spacy.load('en_core_web_sm')
fwd['review'] = fwd['review'].apply(lambda x: str(x))
fwd['entities'] = fwd['review'].apply(lambda x: str(nlp(x).ents))

In [33]:
fwd['sentiment'] = fwd['review'].apply(lambda x: nlp(x).sentiment)

In [52]:
fwd.to_csv('trustpilot_reviews.csv')

In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")

model = AutoModelForSequenceClassification.from_pretrained("yangheng/deberta-v3-base-absa-v1.1")

/Users/SidharrthNagappan/.virtualenvs/multitask-bert/lib/python3.8/site-packages/transformers/convert_slow_tokenizer.py:434: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [46]:
# pipeline = transformers.pipeline('sentiment-analysiss', model=model, tokenizer=tokenizer)
# pipeline('I love this product and the pancake is very good.')
tokenized = tokenizer('I love this product and the pancake is very bad. I am happy with the weather.', return_tensors='pt')
input_ids = tokenized["input_ids"][0]
outputs = model(**tokenized)

In [56]:
from pyabsa.functional import ATEPCCheckpointManager

fwd['review'] = fwd['review'].apply(lambda x: str(x))
aspect_extractor = ATEPCCheckpointManager.get_aspect_extractor(checkpoint='english')
review_aspects = aspect_extractor.extract_aspect(inference_source=fwd['review'].tolist(),
                                               save_result=True,
                                               pred_sentiment=True,  # Predict the sentiment of extracted aspect terms
                                               )

  0%|          | 0/75 [00:00<?, ?it/s, extracting aspect terms...]/Users/SidharrthNagappan/.virtualenvs/multitask-bert/lib/python3.8/site-packages/transformers/models/deberta_v2/modeling_deberta_v2.py:542: DeprecationWarning: `np.int` is a deprecated alias for the builtin `int`. To silence this warning, use `int` by itself. Doing this will not modify any behavior and is safe. When replacing `np.int`, you may wish to use e.g. `np.int64` or `np.int32` to specify the precision. If you wish to review your current use, check the release note link for additional information.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  bucket_pos = np.where(abs_pos <= mid, relative_pos, log_pos * sign).astype(np.int)
  0%|          | 0/102 [00:00<?, ?it/s, classifying aspect sentiments...]/Users/SidharrthNagappan/.virtualenvs/multitask-bert/lib/python3.8/site-packages/pyabsa/core/atepc/prediction/aspect_extractor.py:408: UserWarni

The results of aspect term extraction have been saved in /Users/SidharrthNagappan/Downloads/trustpilot-scraper-master/atepc_inference.result.json
Example 0: NOT <USER:Negative Confidence:0.9997324347496033> FRIENDLY
Example 1: Smooth transaction via online renewal
Example 2: nan
Example 3: nan
Example 4: It was straight forward and the <discount:Positive Confidence:0.9997144341468811> given entice me to take up the policy . With a small sum , I was able to cover for any emergency while driving to and within Malaysia . Will defintely encourage my friends to consider FWD for their insurance needs . Thank you FWD ! !
Example 5: App hang when i was trying to checkout . . . . told me " sorry try again "
Example 6: Easy and fuss - free way to get <travel:Positive Confidence:0.9995972514152527> insurance
Example 7: Fast and easy to understand . <Process:Positive Confidence:0.9998412132263184> and <procedures:Positive Confidence:0.9998428821563721> simple to do . b
Example 8: Called in to make

In [59]:
import json

# dump the result to a json file
with open('review_aspects.json', 'w') as f:
    json.dump(review_aspects, f)

In [63]:
# find most common aspects
from collections import Counter

aspects = set()
for review in review_aspects:
    aspects.update([aspect for aspect in review['aspect']])

In [73]:
aspect_sentiments = {}

for review in review_aspects:
    aspects_list = review['aspect']
    for i in range(len(review['aspect'])):
        aspect = aspects_list[i].lower()
        if aspect not in aspect_sentiments:
            aspect_sentiments[aspect] = [review['sentiment'][i]]
        else:
            aspect_sentiments[aspect].append(review['sentiment'][i])        

In [74]:
aspect_sentiments

{'user': ['Negative',
  'Positive',
  'Negative',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Negative',
  'Positive',
  'Negative',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive'

In [84]:
aspect_sentiments

{'use': ['Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Negative',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',

In [91]:
aspect_sentiments

{'use': ['Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Negative',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',
  'Positive',

### Pain Points

In [94]:
aspect_sentiments.keys()

dict_keys(['use', 'service', 'price', 'process', 'website', 'application', 'coverage', 'insurance', 'navigate', 'policy', 'experience', 'interface', 'pricing', 'staff', 'payment', 'purchase', 'discount', 'apply', 'customer service', 'transaction'])

In [105]:
sentiment_ratios = {}

for aspect in aspect_sentiments:
    pos = aspect_sentiments[aspect].count('Positive')
    neg = aspect_sentiments[aspect].count('Negative')
    neu = aspect_sentiments[aspect].count('Neutral')
    sentiment_ratios[aspect] = {
        'Positive': pos,
        'Negative': neg,
        'Neutral': neu,
    }

In [106]:
restructured_dist = {
    'Positive': {},
    'Negative': {},
    'Neutral': {},
}
for key in sentiment_ratios:
    for sentiment in sentiment_ratios[key]:
        restructured_dist[sentiment][key] = sentiment_ratios[key][sentiment]

In [107]:
px.bar(restructured_dist, barmode='stack')

From the ranking, most aspects have a generally positive sentiment. However, payment remains to be perceived as **problematic**.